In [3]:
from bs4 import BeautifulSoup

with open("bjobs-doc-table.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [11]:
list(soup.tbody.children)

['\n',
 <tr>
 <td headers="d659486e548">jobid</td>
 <td headers="d659486e551">7</td>
 <td headers="d659486e554">id</td>
 <td headers="d659486e557"> </td>
 <td headers="d659486e560" rowspan="21">Common</td>
 </tr>,
 '\n',
 <tr data-hd-product="lsf10.1.0.2">
 <td headers="d659486e548">jobindex</td>
 <td headers="d659486e551">8</td>
 <td headers="d659486e554"> </td>
 <td headers="d659486e557"> </td>
 </tr>,
 '\n',
 <tr>
 <td headers="d659486e548">stat</td>
 <td headers="d659486e551">5</td>
 <td headers="d659486e554"> </td>
 <td headers="d659486e557"> </td>
 </tr>,
 '\n',
 <tr>
 <td headers="d659486e548">user</td>
 <td headers="d659486e551">7</td>
 <td headers="d659486e554"> </td>
 <td headers="d659486e557"> </td>
 </tr>,
 '\n',
 <tr data-hd-product="lsf913">
 <td headers="d659486e548">user_group</td>
 <td headers="d659486e551">15</td>
 <td headers="d659486e554">ugroup</td>
 <td headers="d659486e557"> </td>
 </tr>,
 '\n',
 <tr>
 <td headers="d659486e548">queue</td>
 <td headers="d659486e55

In [22]:
x = list(soup.tbody.children)[0]

In [24]:
x.name

In [78]:
rows = []

for row in soup.tbody.find_all("tr"):
    if row.name != "tr":
        print("eww")
        continue

    cols = list(row.find_all("td"))
    if "pend_reason" in row:
        print(row)

    #     Flattening is problematic if there's an explanation in the cell as well
    #     cells = [x for cell in cols for x in cell.contents]
    cells = [cell.contents[0] for cell in cols]


    def parse(tag, as_int=False, one_word=False):
        s = tag.string
        if s == "\xa0":
            s = None

        if as_int and s:
            s = int(s)

        if one_word:
            assert s is not None, f"Empty cell: {tag}"
            words = s.strip().split(" ")
            #             assert len(words) == 1, words
            s = words[0]

        return s


    print(cells[0])
    parsed_row = {
        "name": parse(cells[0], one_word=True),
        "width": parse(cells[1].string, as_int=True),
        "aliases": parse(cells[2]),
        "unit": parse(cells[3]),
    }
    if cells[0] == "pend_reason":
        print(row)
        badrow = row

    rows.append(parsed_row)

jobid
jobindex
stat
user
user_group
queue
job_name
job_description
proj_name
application
service_class
job_group
job_priority
rsvid
esub
kill_reason
suspend_reason
resume_reason
kill_issue_host
suspend_issue_host
resume_issue_host
dependency
pend_reason
<tr>
<td headers="d659486e548">pend_reason<p>This displays the single key pending reason, including custom
        messages, regardless of the default pending reason level as specified in the
        <span class="keyword parmname">LSB_BJOBS_PENDREASON_LEVEL</span> parameter in the <span class="ph filepath">lsf.conf</span>
        file.</p>
</td>
<td headers="d659486e551">11</td>
<td headers="d659486e554"> </td>
<td headers="d659486e557"> </td>
</tr>
charged_saap
command
pre_exec_command
post_exec_command
resize_notification_command
pids
exit_code
exit_reason
interactive
from_host
first_host
exec_host
nexec_host
        
ask_hosts
alloc_slot
nalloc_slot
host_file
exclusive
nreq_slot
submit_time
start_time
estimated_start_time
specified_s

In [60]:
cols = list(badrow.find_all("td"))
# [cell.contents for cell in cols]

[['pend_reason',
  <p>This displays the single key pending reason, including custom
          messages, regardless of the default pending reason level as specified in the
          <span class="keyword parmname">LSB_BJOBS_PENDREASON_LEVEL</span> parameter in the <span class="ph filepath">lsf.conf</span>
          file.</p>,
  '\n'],
 ['11'],
 ['\xa0'],
 ['\xa0']]

In [54]:
[x for cell in cols for x in cell.contents]

<tr>
<td headers="d659486e548">pend_reason<p>This displays the single key pending reason, including custom
        messages, regardless of the default pending reason level as specified in the
        <span class="keyword parmname">LSB_BJOBS_PENDREASON_LEVEL</span> parameter in the <span class="ph filepath">lsf.conf</span>
        file.</p>
</td>
<td headers="d659486e551">11</td>
<td headers="d659486e554"> </td>
<td headers="d659486e557"> </td>
</tr>

In [79]:
rows

[{'name': 'jobid', 'width': 7, 'aliases': 'id', 'unit': None},
 {'name': 'jobindex', 'width': 8, 'aliases': None, 'unit': None},
 {'name': 'stat', 'width': 5, 'aliases': None, 'unit': None},
 {'name': 'user', 'width': 7, 'aliases': None, 'unit': None},
 {'name': 'user_group', 'width': 15, 'aliases': 'ugroup', 'unit': None},
 {'name': 'queue', 'width': 10, 'aliases': None, 'unit': None},
 {'name': 'job_name', 'width': 10, 'aliases': 'name', 'unit': None},
 {'name': 'job_description',
  'width': 17,
  'aliases': 'description',
  'unit': None},
 {'name': 'proj_name', 'width': 11, 'aliases': 'proj, project', 'unit': None},
 {'name': 'application', 'width': 13, 'aliases': 'app', 'unit': None},
 {'name': 'service_class', 'width': 13, 'aliases': 'sla', 'unit': None},
 {'name': 'job_group', 'width': 10, 'aliases': 'group', 'unit': None},
 {'name': 'job_priority', 'width': 12, 'aliases': 'priority', 'unit': None},
 {'name': 'rsvid', 'width': 40, 'aliases': None, 'unit': None},
 {'name': 'esub',

In [81]:
import json

with open("bjobs_formats.py", "w") as f:
    f.write("bjobs_formats = " + str(rows))

## Field types

In [2]:
data = [
    {
        "jobid": "207744120",
        "jobindex": "0",
        "stat": "RUN",
        "user": "vvolhejn",
        "user_group": "",
        "queue": "normal.4h",
        "job_name": "sleep 300",
        "job_description": "",
        "proj_name": "default",
        "application": "",
        "service_class": "",
        "job_group": "",
        "job_priority": "50",
        "rsvid": "",
        "esub": "-",
        "kill_reason": "",
        "dependency": "",
        "pend_reason": "",
        "charged_saap": "/public_normal/common/vvolhejn",
        "command": "sleep 300",
        "pre_exec_command": "",
        "post_exec_command": "",
        "resize_notification_command": "",
        "pids": "27824,27960,27962",
        "exit_code": "",
        "exit_reason": "",
        "interactive": "N",
        "from_host": "eu-login-21",
        "first_host": "eu-c7-112-03",
        "exec_host": "eu-c7-112-03",
        "nexec_host": "1",
        "alloc_slot": "eu-c7-112-03",
        "nalloc_slot": "1",
        "host_file": "",
        "exclusive": "N",
        "nreq_slot": "1",
        "submit_time": "Mar  9 17:00",
        "start_time": "Mar  9 17:01",
        "estimated_start_time": "",
        "specified_start_time": "",
        "specified_terminate_time": "",
        "time_left": "3:58 L",
        "finish_time": "Mar  9 21:01 L",
        "estimated_run_time": "",
        "ru_utime": "0",
        "ru_stime": "0",
        "%complete": "0.67% L",
        "warning_action": "",
        "action_warning_time": "",
        "pendstate": "NOTPEND",
        "pend_time": "44",
        "ependtime": "44",
        "ipendtime": "0",
        "estimated_sim_start_time": "",
        "effective_plimit": "",
        "plimit_remain": "",
        "effective_eplimit": "",
        "eplimit_remain": "",
        "cpu_used": "",
        "run_time": "96 second(s)",
        "idle_factor": "",
        "exception_status": "",
        "slots": "1",
        "mem": "2 Mbytes",
        "max_mem": "2 Mbytes",
        "avg_mem": "2 Mbytes",
        "memlimit": "3 G",
        "swap": "0 Mbytes",
        "swaplimit": "",
        "gpu_num": "",
        "gpu_mode": "",
        "j_exclusive": "",
        "gpu_alloc": "",
        "nthreads": "4",
        "hrusage": [],
        "min_req_proc": "",
        "max_req_proc": "",
        "effective_resreq": "select[((mem<597000&&!xmpp&&!mpp&&!par&&(centos>=7)&&(stable||beta||delta||gamma)) && (type == any)) && ((sssd&&scratch>1000&&lustre) && (type == any))] order[-sload] rusage[mem=1024.00] same[model:centos:ibfabric] affinity[core(1)*1:cpubind=core:distribute=pack] ",
        "combined_resreq": "select[((mem<597000&&!xmpp&&!mpp&&!par&&(centos>=7)&&(stable||beta||delta||gamma)) && (sssd&&scratch>1000&&lustre)) && (type == any)] order[-sload] rusage[mem=1024.00] same[model:centos:ibfabric] affinity[core(1)*1:cpubind=core:distribute=pack]",
        "network_req": "",
        "filelimit": "",
        "corelimit": "",
        "stacklimit": "",
        "processlimit": "",
        "runtimelimit": "240.0",
        "plimit": "",
        "eplimit": "",
        "input_file": "",
        "output_file": "lsf.o207744120",
        "error_file": "",
        "output_dir": "",
        "sub_cwd": "$HOME",
        "exec_home": "/cluster/home/vvolhejn",
        "exec_cwd": "/cluster/home/vvolhejn",
        "licproject": "",
        "forward_cluster": "",
        "forward_time": "",
        "srcjobid": "207744120",
        "dstjobid": "",
        "source_cluster": "euler",
        "energy": "",
        "gpfsio": "",
    }
][0]

In [12]:
def bjobs_bool(s):
    d = {
        "N": False,
        "Y": True,
    }
    return d[s]

In [9]:
types = {
    "jobid": int,
    "jobindex": int,
    "job_priority": int,
    "nexec_host": int,
    "nalloc_slot": int,
    "nreq_slot": int,
    "ru_utime": int,
    "ru_stime": int,
    "pend_time": int,
    "ependtime": int,
    "ipendtime": int,
    "slots": int,
    "nthreads": int,
    "runtimelimit": float,
    "srcjobid": int,
}

In [11]:
for k, v in data.items():
    if k in types.keys():
        # Just make sure the conversion works
        v = types[k](v)
        continue

    try:
        float(v)
        if "." in v:
            print(f'"{k}": float,')
        else:
            print(f'"{k}": int,')
    except:
        print(f"No type for {k}: {v}")

No type for stat: RUN
No type for user: vvolhejn
No type for user_group: 
No type for queue: normal.4h
No type for job_name: sleep 300
No type for job_description: 
No type for proj_name: default
No type for application: 
No type for service_class: 
No type for job_group: 
No type for rsvid: 
No type for esub: -
No type for kill_reason: 
No type for dependency: 
No type for pend_reason: 
No type for charged_saap: /public_normal/common/vvolhejn
No type for command: sleep 300
No type for pre_exec_command: 
No type for post_exec_command: 
No type for resize_notification_command: 
No type for pids: 27824,27960,27962
No type for exit_code: 
No type for exit_reason: 
No type for interactive: N
No type for from_host: eu-login-21
No type for first_host: eu-c7-112-03
No type for exec_host: eu-c7-112-03
No type for alloc_slot: eu-c7-112-03
No type for host_file: 
No type for exclusive: N
No type for submit_time: Mar  9 17:00
No type for start_time: Mar  9 17:01
No type for estimated_start_time: 